In [1]:
# Install pyspark either via pip install or through anaconda
import pyspark

In [2]:
# Check pyspark version
pyspark.__version__

'3.0.0'

In [52]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnull, when, count, col
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

### Note:

In order to start a spark session, you must have Java installed on your computer. 

Install Java from the Oracle's download site, chose the MacOS installer, clicked on `jdk-13.0.2_osx-x64_bin.dmg`

Download site: https://www.oracle.com/java/technologies/javase-jdk13-downloads.html

In [8]:
spark = SparkSession \
    .builder \
    .appName('Practice PySpark Classification') \
    .getOrCreate()

In [9]:
spark

In [11]:
dataset = (spark.read
          .format("csv")
          .option('header', 'true')
          .load("titanic/train.csv"))

In [14]:
# the head function does not work the same way as in Pandas
dataset.head()

Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S')

In [13]:
dataset.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [15]:
# Only do this if the dataset is small enough. This gets you all the data at once
dataset.toPandas()

PassengerId Survived Pclass  \
0             1        0      3   
1             2        1      1   
2             3        1      3   
3             4        1      1   
4             5        0      3   
..          ...      ...    ...   
886         887        0      2   
887         888        1      1   
888         889        0      3   
889         890        1      1   
890         891        0      3   

                                                  Name     Sex   Age SibSp  \
0                              Braund, Mr. Owen Harris    male    22     1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female    38     1   
2                               Heikkinen, Miss. Laina  female    26     0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female    35     1   
4                             Allen, Mr. William Henry    male    35     0   
..                                                 ...     ...   ...   ...   
886                              Montvila, Rev. Juozas    male    27     0   
887                       Graham, Miss. Margaret Edith  female    19     0   
888       "Johnston, Miss. Catherine Helen ""Carrie"""  female  None     1   
889                              Behr, Mr. Karl Howell    male    26     0   
890                                Dooley, Mr. Patrick    male    32     0   

    Parch            Ticket     Fare Cabin Embarked  
0       0         A/5 21171     7.25  None        S  
1       0          PC 17599  71.2833   C85        C  
2       0  STON/O2. 3101282    7.925  None        S  
3       0            113803     53.1  C123        S  
4       0            373450     8.05  None        S  
..    ...               ...      ...   ...      ...  
886     0            211536       13  None        S  
887     0            112053       30   B42        S  
888     2        W./C. 6607    23.45  None        S  
889     0            111369       30  C148        C  
890     0            370376     7.75  None        Q  

[891 rows x 12 columns]

## Exploratory Data Analysis

In [17]:
dataset.count()

891

In [18]:
dataset.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [19]:
dataset.dtypes

[('PassengerId', 'string'),
 ('Survived', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'string'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'string'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [21]:
dataset.describe().toPandas()

summary        PassengerId             Survived              Pclass  \
0   count                891                  891                 891   
1    mean              446.0   0.3838383838383838   2.308641975308642   
2  stddev  257.3538420152301  0.48659245426485753  0.8360712409770491   
3     min                  1                    0                   1   
4     max                 99                    1                   3   

                                               Name     Sex  \
0                                               891     891   
1                                              None    None   
2                                              None    None   
3  "Andersson, Mr. August Edvard (""Wennerstrom"")"  female   
4                       van Melkebeke, Mr. Philemon    male   

                  Age               SibSp                Parch  \
0                 714                 891                  891   
1   29.69911764705882  0.5230078563411896  0.38159371492704824   
2  14.526497332334035  1.1027434322934315   0.8060572211299488   
3                0.42                   0                    0   
4                   9                   8                    6   

               Ticket               Fare Cabin Embarked  
0                 891                891   204      889  
1  260318.54916792738   32.2042079685746  None     None  
2  471609.26868834975  49.69342859718089  None     None  
3              110152                  0   A10        C  
4           WE/P 5735               93.5     T        S

## Data preparation

In [23]:
# Cast columns into their correct types
dataset = dataset.select(col('Survived').cast('float'),
                         col('Pclass').cast('float'),
                         col('Sex'),
                         col('Age').cast('float'),
                         col('Fare').cast('float'),
                         col('Embarked')
                        )

In [24]:
dataset.show(10)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
|     0.0|   3.0|  male|35.0|   8.05|       S|
|     0.0|   3.0|  male|null| 8.4583|       Q|
|     0.0|   1.0|  male|54.0|51.8625|       S|
|     0.0|   3.0|  male| 2.0| 21.075|       S|
|     1.0|   3.0|female|27.0|11.1333|       S|
|     1.0|   2.0|female|14.0|30.0708|       C|
+--------+------+------+----+-------+--------+
only showing top 10 rows



In [26]:
# Check for null values
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



In [27]:
# Drop rows with null values
dataset = dataset.replace('?', None).dropna(how='any')

In [28]:
dataset.show(10)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
|     0.0|   3.0|  male|35.0|   8.05|       S|
|     0.0|   1.0|  male|54.0|51.8625|       S|
|     0.0|   3.0|  male| 2.0| 21.075|       S|
|     1.0|   3.0|female|27.0|11.1333|       S|
|     1.0|   2.0|female|14.0|30.0708|       C|
|     1.0|   3.0|female| 4.0|   16.7|       S|
+--------+------+------+----+-------+--------+
only showing top 10 rows



In [29]:
dataset.count()

712

In [31]:
# The Spark ML library only works with numeric data. Hence, we need to encode the string columns

dataset = StringIndexer(
    inputCol='Sex', 
    outputCol='Gender', 
    handleInvalid='keep').fit(dataset).transform(dataset)

dataset = StringIndexer(
    inputCol='Embarked', 
    outputCol='Boarded', 
    handleInvalid='keep').fit(dataset).transform(dataset)

In [32]:
dataset.show(10)

+--------+------+------+----+-------+--------+------+-------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|Boarded|
+--------+------+------+----+-------+--------+------+-------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|    0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|    1.0|
|     1.0|   3.0|female|26.0|  7.925|       S|   1.0|    0.0|
|     1.0|   1.0|female|35.0|   53.1|       S|   1.0|    0.0|
|     0.0|   3.0|  male|35.0|   8.05|       S|   0.0|    0.0|
|     0.0|   1.0|  male|54.0|51.8625|       S|   0.0|    0.0|
|     0.0|   3.0|  male| 2.0| 21.075|       S|   0.0|    0.0|
|     1.0|   3.0|female|27.0|11.1333|       S|   1.0|    0.0|
|     1.0|   2.0|female|14.0|30.0708|       C|   1.0|    1.0|
|     1.0|   3.0|female| 4.0|   16.7|       S|   1.0|    0.0|
+--------+------+------+----+-------+--------+------+-------+
only showing top 10 rows



In [33]:
dataset.dtypes

[('Survived', 'float'),
 ('Pclass', 'float'),
 ('Sex', 'string'),
 ('Age', 'float'),
 ('Fare', 'float'),
 ('Embarked', 'string'),
 ('Gender', 'double'),
 ('Boarded', 'double')]

In [34]:
# Drop unnecessary columns
dataset = dataset.drop('Sex')
dataset = dataset.drop('Embarked')

In [35]:
dataset.show(10)

+--------+------+----+-------+------+-------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|
+--------+------+----+-------+------+-------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|
|     0.0|   1.0|54.0|51.8625|   0.0|    0.0|
|     0.0|   3.0| 2.0| 21.075|   0.0|    0.0|
|     1.0|   3.0|27.0|11.1333|   1.0|    0.0|
|     1.0|   2.0|14.0|30.0708|   1.0|    1.0|
|     1.0|   3.0| 4.0|   16.7|   1.0|    0.0|
+--------+------+----+-------+------+-------+
only showing top 10 rows



In [37]:
# PySpark predicts with a column with all the features smashed together into a list-like structure. 
# Hence, we need to assemble the features

required_features = ['Pclass', 'Age', 'Fare', 'Gender', 'Boarded']
assembler = VectorAssembler(inputCols=required_features, outputCol='features')
transformed_data = assembler.transform(dataset)

In [38]:
transformed_data.show(10)

+--------+------+----+-------+------+-------+--------------------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|            features|
+--------+------+----+-------+------+-------+--------------------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|[3.0,22.0,7.25,0....|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|[1.0,38.0,71.2833...|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|[3.0,26.0,7.92500...|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|[1.0,35.0,53.0999...|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|[3.0,35.0,8.05000...|
|     0.0|   1.0|54.0|51.8625|   0.0|    0.0|[1.0,54.0,51.8624...|
|     0.0|   3.0| 2.0| 21.075|   0.0|    0.0|[3.0,2.0,21.07500...|
|     1.0|   3.0|27.0|11.1333|   1.0|    0.0|[3.0,27.0,11.1332...|
|     1.0|   2.0|14.0|30.0708|   1.0|    1.0|[2.0,14.0,30.0708...|
|     1.0|   3.0| 4.0|   16.7|   1.0|    0.0|[3.0,4.0,16.70000...|
+--------+------+----+-------+------+-------+--------------------+
only showing top 10 rows



## Model training

In [44]:
# train test split
(train, test) = transformed_data.randomSplit([0.75,0.25])

### Logistic Regression

In [45]:
lr = LogisticRegression(labelCol='Survived', featuresCol='features')

In [47]:
lr_model = lr.fit(train)

In [48]:
lr_pred = lr_model.transform(test)

### Random Forest

In [49]:
rf = RandomForestClassifier(labelCol='Survived', featuresCol='features')

In [50]:
rf_model = rf.fit(train)

In [51]:
rf_pred = rf_model.transform(test)

## Model evaluation

In [55]:
# Accuracy
accuracy = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')
# Precision
precision = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='weightedPrecision')
# Recall
recall = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='weightedRecall')

### Logistic Regression

In [56]:
print('Logistic Regression Accuracy:', accuracy.evaluate(lr_pred))
print('Logistic Regression Precision:', precision.evaluate(lr_pred))
print('Logistic Regression Recall:', recall.evaluate(lr_pred))

Logistic Regression Accuracy: 0.7967032967032966
Logistic Regression Precision: 0.7962787212787212
Logistic Regression Recall: 0.7967032967032968


### Random Forest

In [57]:
print('Random Forest Accuracy:', accuracy.evaluate(rf_pred))
print('Random Forest Precision:', precision.evaluate(rf_pred))
print('Random Forest Recall:', recall.evaluate(rf_pred))

Random Forest Accuracy: 0.8296703296703297
Random Forest Precision: 0.8287559963256557
Random Forest Recall: 0.8296703296703296
